# Embedding training and testing data sets

In [1]:
%reset -f
import ipdb

import logging
import sys

my_stderr = sys.stderr = open('errors.txt', 'w')  # redirect stderr to file
get_ipython().log.handlers[0].stream = my_stderr  # log errors to new stderr
get_ipython().log.setLevel(logging.INFO)  # errors are logged at info level

import os
import sys
import argparse
import ipdb
from tqdm import tqdm
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import tensorflow_text

In [2]:
# LASER="LASER-master"
mldir = "ml-training/SECRECY-EN/data_hard_negatives" # training data and embeddings are held here
model_name = "SECRECY-EN"
edir = mldir + "/embed_english" # location of laser script that that embeds to LASER SS
#embedding_name = 'USE'
embedding_name = 'USEm'

In [3]:
def USE(txt_list, use_embedder):
    output = []
    txt_df = pd.Series(txt_list)
    batch_size = 12
    for k,g in tqdm(txt_df.groupby(np.arange(len(txt_df))//batch_size)):
        output.append(use_embedder(g.tolist()).numpy())
    output = np.concatenate(output, axis=0).flatten()
    #print(output.shape)
    #print(output.min())
    #print(output.max())
    #print('\n')
    return output

In [4]:
# embed train and dev sets

In [ ]:
if not os.path.exists(edir):
    os.mkdir(edir)

#lang="en"
for lang in ['en']:
    print('\nProcessing:')
    embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
    #embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

    for part in ('train', 'dev'):
        print('Starting ' + part)
        # Spanish only exists for test
        if (lang=='es') & (part!='test'):
            continue
        cfname = os.path.join(mldir, part)
        if os.path.exists(cfname + '.' + embedding_name + '.enc.' + lang): # embedding_name
            print(cfname + '.' + embedding_name + '.enc.' + lang + ' Exists')
        txt = []
        
        # copy original files to embed directory, then embed it
        print('Copying txt/lbl files')
        os.system('cp ' + cfname + '.txt.' + lang + ' ' + os.path.join(edir, part) + '.txt.' + lang)
        os.system('cp ' + cfname + '.lbl.' + lang + ' ' + os.path.join(edir, part) + '.lbl.' + lang)
        
        # embed file in embed directory
        with open(cfname + '.txt.' + lang) as f:
            for l in f:
                txt.append(l)
        use_np = USE(txt, embed)
        print(use_np.dtype, use_np.shape)
        use_np.tofile(edir + "/" + part + '.' + embedding_name + '.enc.' + lang)


Processing:
Starting train
Copying txt/lbl files


In [ ]:
# for test, split into smaller files
# split -l 1000 -d -a 3 test.txt.en test.txt.en_
# mkdir test_txt_en
# mv test.txt.en_* test_txt_en/

# split -l 1000 -d -a 3 test.txt.es test.txt.es_
# mkdir test_txt_es
# mv test.txt.es_* test_txt_es/

In [ ]:
if not os.path.exists(edir):
    os.mkdir(edir)

#lang="en"
for lang in ['en']:
    print('\nProcessing:')
    embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
    #embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

    for part in ['test']:
        cfname = mldir + '/test_txt_en/test.txt.en_'
        
        # copy original files to embed directory, then embed it
        print('Copying txt/lbl files')
        os.system('cp ' + mldir + '/test.txt.en' + ' ' + os.path.join(edir, part) + '.txt.' + lang)
        os.system('cp ' + mldir + '/test.lbl.en' + ' ' + os.path.join(edir, part) + '.lbl.' + lang)
        
        # embed file in embed directory
        for i in range(692):
            txt = []
            if i>=100:
                filename = cfname + str(i)
            if (i>=10) & (i<=99):
                filename = cfname + '0' + str(i)
            if i<=9:
                filename = cfname + '00' + str(i)
            print('Reading from: ' + filename)
            with open(filename) as f:
                for l in f:
                    txt.append(l)
            use_np = USE(txt, embed)
            #print(use_np.dtype, use_np.shape)
            print('Writing to: ' + edir + "/" + part + '.' + embedding_name + '.enc.' + lang + '_' + str(i))
            print('\n')
            use_np.tofile(edir + "/" + part + '.' + embedding_name + '.enc.' + lang + '_' + str(i))
